In [1]:
from utils import *

In [2]:
pool = Pool(4)

In [3]:
print(f"g0: {g0}\no0: {o0} \nka: {ka}")

NameError: name 'g0' is not defined

In [ ]:
# rotating
g0 = 100
ka = 0.1
b = 1
o0 = 20

In [8]:
# fixed
g0 = 100
ka = 4
b = 1
o0 = 1

In [ ]:
# rotating
alpha=0.03
beta = 0.2
ka = g0 * alpha**2;o0 = g0 * beta
g0 = 100;
b = 1

In [6]:
# fixed
alpha=0.2
beta = 0.01
ka = g0 * alpha**2;o0 = g0 * beta
g0 = 100;
b = 1

In [9]:
den = (g0*ka+o0**2)
print(f"g0 ka + o0^2 = {den} >> {o0**2} = o0**2   \t ratio: {den//o0**2}")
print(f"g0 ka + o0^2 = {den} >> {ka**2:0.3f} = ka**2   \t ratio: {den//ka**2}")
print(f"g0 ka/o0^2 = {g0*ka/o0**2} ")
relaxation_time = (g0*ka + o0**2) * max([2/ka**2/g0, 1/ka*o0**2])
print(f"relaxation time (rotating): {2/(ka**2*g0)*den:0.3f}")
print(f"relaxation time (fixed):    {1/(ka*o0**2)*den}")

g0 ka + o0^2 = 401 >> 1 = o0**2   	 ratio: 401
g0 ka + o0^2 = 401 >> 16.000 = ka**2   	 ratio: 25
g0 ka/o0^2 = 400.0 
relaxation time (rotating): 0.501
relaxation time (fixed):    100.25


In [10]:
N=5000000
warmup=0
dt=0.001
samples=5
skip=1
runs=4
time = np.linspace(0,N//skip *dt, N//skip)

R,S,H,_ = get_RSH_harmonic_onlyka(o0**2,g0,b,ka)

(len([1]) * N * runs * samples *4 // skip) * 4 / 1024**3

1.4901161193847656

In [11]:
sigma = get_cov(np.array([R]), np.array([S]), np.array([H]))

x0,y0,vx0,vy0 = np.random.multivariate_normal(
    [0,0,0,0], 0*sigma[...,0], samples).T
kwargs = dict(x0=x0,y0=y0,vx0=vx0,vy0=vy0,
    N=N, samples=samples,
    dt=dt, warmup=warmup, skip=skip,
    omega0=o0,
    gamma0=g0,b=b, kappa=ka)
result = simulate_2d_only_memory(pool,runs,**kwargs)

result = np.array(result); result.shape

(6, 20, 5000000)

### Start Analisys

In [ ]:
cov_all = np.einsum("akj,bkj->jab", result[:4,:,:],result[:4,:,:],optimize=True)/(result.shape[1]-1)
# cov = cov[df[df.b].index]

cov=cov_all[:]

In [ ]:
sp = 1000
cov_R = cov[::sp,(0,1),(0,1)].transpose(0,1).reshape(-1,2)
cov_S = cov[::sp,(2,3),(2,3)].transpose(0,1).reshape(-1, 2)
cov_H = cov[::sp,(0,1,2,3),(3,2,1,0)].transpose(0,1).reshape(-1, 4)
cov_Z = cov[::sp,(0,0,1,1,2,2,3,3),(1,2,0,3,0,3,1,2)].transpose(0,1).reshape(-1, 8)

In [ ]:
plt.figure(figsize=(14,8))
plt.subplot(2,2,1)
plt.plot(cov_R, alpha=0.2);
plt.subplot(2,2,2)
plt.plot(cov_S, alpha=0.2);
plt.subplot(2,2,3)
plt.plot(cov_H, alpha=0.2);
plt.subplot(2,2,4)
plt.plot(cov_Z, alpha=0.2);

In [ ]:
stationari_treshold = 3000*sp

In [ ]:
result.shape

In [ ]:
x,y,vx,vy,_,_ = result[:,:,stationari_treshold:]

In [ ]:
start = 0*100
rang  = 3*int(2*3.14*1000*den**-0.5)
path = 1
factor = 2.5

In [ ]:
axs = plt.subplots(5,2, figsize=(5,20))[1].flatten()
for path, ax in enumerate(axs):
    path+=10
    cm = plt.get_cmap("Greys")
    for i in range(rang-1):
        ax.plot(x[path][start:][i:i+2],y[path][start:][i:i+2],  c=cm(0.2+0.8*i/rang))
        ax.set_aspect("equal")
        ax.set_xlim(-factor*R**0.5,factor*R**0.5)
        ax.set_ylim(-factor*R**0.5,factor*R**0.5)        

##### Plots for rotation regime

##### Plots for fixed regime

In [ ]:
cm(0.1)

In [ ]:
cm.from_list([1,2,3])

In [ ]:
plt.scatter(x[path][start:][:rang],y[path][start:][:rang], c=cm(0.2+0.8*i/rang))

In [ ]:
df["Rx"], df["Rx_err"]= mean_with_err(Rx,axis=-1)
df["Ry"], df["Ry_err"]= mean_with_err(Ry,axis=-1)
df["Sx"], df["Sx_err"]= mean_with_err(Sx,axis=-1)
df["Sy"], df["Sy_err"]= mean_with_err(Sy,axis=-1)
df["Hx"], df["Hx_err"]= mean_with_err(Hx,axis=-1)
df["Hy"], df["Hy_err"]= mean_with_err(Hy,axis=-1)

# df["Rx"], df["Rx_err"]= mean_with_err(Rx,axis=-1)


In [ ]:
gb=df.groupby(["A","B","C","D","F"])

plt.figure(figsize=(10,7), dpi=150)
plt.title("H vs b")
plt.xlabel("b");plt.ylabel("H")
plt.plot(bs_expected, H_expected,ls=(0, (5, 10)), label="harmonic (theory)")
for key in gb.groups:
    gp = gb.get_group(key)
#     plt.plot(gp.b, gp.Hx)
    plt.errorbar(gp.b, gp.Hx, yerr=gp.Hx_err, capsize=7, marker="o", ls="none", 
                 label=f"A: {gp.A.mean():0.2f} B: {gp.B.mean():0.2f} C: {gp.C.mean():0.2f}")
# plt.xlim(-1,5)
# plt.ylim(-4,1)
plt.legend()

In [ ]:
gb=df.groupby(["A","B","C","D","F"])

plt.figure(figsize=(10,7), dpi=150)
plt.subplot(121)
plt.title("R vs b")
plt.xlabel("b");plt.ylabel("R")
# plt.plot(*zip(*[(b, get_RSH(-7, 0,100,b,10)[0]) for b in np.linspace(min(bs), max(bs))]),ls=(0, (5, 10)), label="harmonic (theory)")
for key in gb.groups:
    gp = gb.get_group(key)
#     plt.plot(gp.b, gp.Hx)
    plt.errorbar(gp.b, gp.Rx, yerr=gp.Rx_err, capsize=7, marker="o", ls="none", 
                label=f"A: {gp.A.mean():0.2f} B: {gp.B.mean():0.2f} C: {gp.C.mean():0.2f}")
plt.minorticks_on()
plt.legend()

plt.subplot(122)
plt.title("S vs b")
plt.xlabel("b");plt.ylabel("S")
plt.plot(bs_expected, bs_expected**2/2/0.3+5)
# plt.plot(*zip(*[(b, get_RSH(-7, 0,100,b,10)[1]) for b in np.linspace(min(bs), max(bs))]),ls=(0, (5, 10)), label="harmonic (theory)")
for key in gb.groups:
    gp = gb.get_group(key)
#     plt.plot(gp.b, gp.Hx)
    plt.errorbar(gp.b, gp.Sx, yerr=gp.Sx_err, capsize=7, marker="o", ls="none", 
                label=f"A: {gp.A.mean():0.2f} B: {gp.B.mean():0.2f} C: {gp.C.mean():0.2f}")
plt.legend()

## xry analysis

In [ ]:
res = results[...,stationari_treshold::]

uxs, uys = [], []
for row in df.itertuples():
    ux, uy = Uxy_2(row.A, row.B, row.C, row.D, row.F, res[row.Index,0].flatten(),res[row.Index,1].flatten() )
    uxs.append(ux.reshape(res.shape[2], res.shape[3]))
    uys.append(uy.reshape(res.shape[2], res.shape[3]))

uxs=np.array(uxs)
uys=np.array(uys)

xry= res[:,0] * res[:,5]
yrx= res[:,1] * res[:,4]

uxry= uxs*res[:,5]
uyrx= uys*res[:,4]

In [ ]:
xry.shape

In [ ]:
df["xry"], df["xry_err"]=mean_with_err(xry.reshape(xry.shape[0],-1), axis=1)
df["uxry"], df["uxry_err"]=mean_with_err(uxry.reshape(uxry.shape[0],-1), axis=1)

In [ ]:
gb=df.groupby(["A","B","C","D","F"])

plt.figure(figsize=(15,10), dpi=150)
plt.title("xry vs b")
plt.xlabel("b");plt.ylabel("xry")
# plt.plot(bs_expected, H_expected,ls=(0, (5, 10)), label="harmonic (theory)")
for key in gb.groups:
    gp = gb.get_group(key)
#     plt.plot(gp.b, gp.Hx)
    plt.errorbar(gp.b, gp.xry, yerr=gp.xry_err, capsize=7, marker="o", ls="none", 
                 label=f"A: {gp.A.mean():0.2f} B: {gp.B.mean():0.2f} C: {gp.C.mean():0.2f}")
plt.grid()
# plt.xlim(-1,5)
# plt.ylim(-4,1)
plt.legend()

In [ ]:
gb=df.groupby(["A","B","C","D","F"])

plt.figure(figsize=(15,10), dpi=150)
plt.title("uxry vs b")
plt.xlabel("b");plt.ylabel("uxry")
# plt.plot(bs_expected, H_expected,ls=(0, (5, 10)), label="harmonic (theory)")
for key in gb.groups:
    gp = gb.get_group(key)
#     plt.plot(gp.b, gp.Hx)
    plt.errorbar(gp.b, gp.uxry, yerr=gp.uxry_err, capsize=7, marker="o", ls="none", 
                 label=f"A: {gp.A.mean():0.2f} B: {gp.B.mean():0.2f} C: {gp.C.mean():0.2f}")
plt.grid()
# plt.xlim(-1,5)
# plt.ylim(-4,1)
plt.legend()

In [ ]:
gp.g0.mean() * gp.ka.mean()**2

In [ ]:
gb=df.groupby(["A","B","C","D","F"])

plt.figure(figsize=(10,7), dpi=150)
plt.title("H vs b")
plt.xlabel("b");plt.ylabel("H")
plt.plot(bs_expected, H_expected,ls=(0, (5, 10)), label="harmonic (theory)")
for ii, key in enumerate(gb.groups):
    
    gp = gb.get_group(key)
    
    g0kasq=gp.g0.mean() * gp.ka.mean()**2
#     plt.plot(gp.b, gp.Hx)
    plt.errorbar(gp.b, gp.Hx, yerr=gp.Hx_err, capsize=7, marker="o",color=COLORS[ii], ls="none", 
                 label=f"A: {gp.A.mean():0.2f} B: {gp.B.mean():0.2f} C: {gp.C.mean():0.2f}")
    
    plt.errorbar(gp.b+0.1, gp.H + gp.uxry/g0kasq, yerr=gp.uxry_err/g0kasq, capsize=7,color=COLORS[ii], marker="s", mfc="white", ls="none", 
                 label=f"theoretical guess")
# plt.xlim(-1,5)
# plt.ylim(-4,1)
plt.legend()

In [ ]:
plt.plot(xry)